In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import time

In [5]:
def func(x, m1, m2, m3, m4, m5, m6):
    return m1 + m2 /(1 + np.exp(m3*m4 -m3*x)) - m2/(1 + np.exp(m5 *m6-m5*x))

#注意初值
def DofittingandValidation(yData, xData):
    Parameters, pcov = curve_fit(func, xData, yData, p0=[5,40,0.1,140,0.1,270], maxfev=80)#,method='trf', maxfev=1000000)
    return Parameters

#注意时间序列
def get_python_param(data_lon_lat,data):

    pars=[]
    for i in range(data.shape[0]):
        lon_lat=data_lon_lat[i]
        yData = data[i,:]
        
        timeDims=yData.size
        xData=4 * (np.array(list(range(timeDims)))) + 1
        
        index=np.where(yData!=-9)
        yData=yData[index]
        xData=xData[index]

        if yData.size==0:
            continue
        else:
            try:
                Parameters, pcov = curve_fit(func, xData, yData, p0=[5,40,0.1,140,0.1,270], maxfev=80)
            except RuntimeError: 
                continue
        pars_ = Parameters 
        x=np.hstack((lon_lat,pars_))
        pars.append(x)
    return np.array(pars)

In [6]:
start_time = time.time()

dataset=pd.read_excel(r'D:\Desktop\mypaper\version20\LAI\remove_outler\LAI_remove_array.xls')#,sheet_name='Sheet1'
# dataset = dataset.set_index(['lat','lon'])
dataset=np.array(dataset)

data_lon_lat=dataset[:,0:2]
data=dataset[:,2::]

param=get_python_param(data_lon_lat,data)
print("Thread pool execution in " + str(time.time() - start_time), "seconds")

writer = pd.ExcelWriter(r"D:\Desktop\mypaper\version20\LAI\remove_outler\LAI_remove_array_param_python_limit_80.xls")

pd.DataFrame(param).to_excel(writer,na_rep=0,index=False,\
                                   header=['lon','lat','p0','p1','p2','p3','p4','p5'])
writer.save()
writer.close()


C:\Users\wangsj\AppData\Local\Temp/ipykernel_16176/2676896609.py:2: RuntimeWarning: overflow encountered in exp
  return m1 + m2 /(1 + np.exp(m3*m4 -m3*x)) - m2/(1 + np.exp(m5 *m6-m5*x))


Thread pool execution in 24.847309827804565 seconds


D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3361: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  if (await self.run_code(code, result,  async_=asy)):
